<a href="https://colab.research.google.com/github/senu2000/Sentiment-Analysis-using-NLP/blob/main/Sentiment_Analysis_using_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd

# Step 1 – Load the Dataset

In [2]:
df = pd.read_csv('/content/Tweets.csv', encoding = 'latin-1')

In [3]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [4]:
df.shape

(14640, 15)

In [5]:
df = df[["airline_sentiment", "text"]]

In [6]:
df.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [24]:
df.groupby('airline_sentiment').describe()

text         \
                  count unique   
airline_sentiment                
negative           9178   9087   
neutral            3099   3067   
positive           2363   2298   

                                                                           \
                                                                 top freq   
airline_sentiment                                                           
negative           @AmericanAir that's 16+ extra hours of travel ...    2   
neutral                                           @SouthwestAir sent    5   
positive                                            @JetBlue thanks!    5   

                  clean_text         \
                       count unique   
airline_sentiment                     
negative                9178   9083   
neutral                 3099   3027   
positive                2363   2262   

                                                                           
                                                                 top freq  
airline_sentiment                                                          
negative           @ americanair fyi ... call still get drop . ho...    2  
neutral            @ jetblu 's ceo battl appeas passeng wall stre...    8  
positive                                            @ jetblu thank !   12

# Step 2 – Preprocess Text

In [11]:
import nltk
import string
import re


from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
from nltk.corpus import stopwords

nltk.download('punkt')
ps = PorterStemmer()


def clean_text(text):
    text = text.lower()
    text = re.sub(r'http.?://[^\s]+[\s]?', '', text)
    text = nltk.word_tokenize(text)
    y = []

    # Remove stopwords
    for i in text:
        if i not in stopwords.words('english'):
            y.append(i)

    # Stem words after filtering stopwords
    stemmed_words = [ps.stem(i) for i in y]

    # Return the cleaned and stemmed text as a single string
    return " ".join(stemmed_words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
df['clean_text'] = df['text'].apply(clean_text)

In [13]:
df.head()

,airline_sentiment,text,clean_text
0,neutral,@VirginAmerica What @dhepburn said.,@ virginamerica @ dhepburn said .
1,positive,@VirginAmerica plus you've added commercials t...,@ virginamerica plu 've ad commerci experi ......
2,neutral,@VirginAmerica I didn't today... Must mean I n...,@ virginamerica n't today ... must mean need t...
3,negative,@VirginAmerica it's really aggressive to blast...,@ virginamerica 's realli aggress blast obnoxi...
4,negative,@VirginAmerica and it's a really big bad thing...,@ virginamerica 's realli big bad thing


# Step 3 – Feature Extraction

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
tf_vec = TfidfVectorizer(max_features=3000)
x = tf_vec.fit_transform(df['clean_text']).toarray()

In [23]:
x.shape

(14640, 3000)

In [16]:
y = df['airline_sentiment'].values

# Step 4 – Train Model

In [17]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

In [18]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(x_train, y_train)

MultinomialNB()

In [19]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score

y_pred = model.predict(x_test)

print(accuracy_score(y_test, y_pred))

0.7219945355191257


In [21]:
from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier()
model.fit(x_train,y_train)

y_pred = model.predict(x_test)
print(accuracy_score(y_test,y_pred))

0.7482923497267759
